#### Gede Indra Adi Brata - 225150200111006

Tugas :

1. Ubah update weight dengan cara Backpropagation
2. Lakukan prediksi dengan dataset iris
3. Tunjukkan accuracy score

In [138]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
import pandas as pd

Architecture

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def bp_fit(train_data, train_target, alpha=0.1, max_epoch=100, max_error=0.5):
    input_dim = train_data.shape[1]   
    hidden_dim = 3                    
    output_dim = train_target.shape[1] 

    w = np.random.random((input_dim, hidden_dim))  
    v = np.random.random((hidden_dim, output_dim))  
    b = np.random.random(hidden_dim + output_dim) 
    epoch = 0
    mse = float('inf')  

    while epoch < max_epoch and mse > max_error:
        mse = 0  
        for data, target in zip(train_data, train_target):
            # Forward pass
            z_in = np.dot(data, w) + b[:hidden_dim]  
            z = sigmoid(z_in)                         
            y_in = np.dot(z, v) + b[hidden_dim:]      
            y = sigmoid(y_in)                         

            error = target - y
            mse += np.sum(error**2)  

            # Backward pass (gradient calculation)
            delta_output = error * sigmoid_derivative(y_in)  
            delta_hidden = np.dot(delta_output, v.T) * sigmoid_derivative(z_in)  

            # Update weights and biases for output layer
            v += alpha * np.outer(z, delta_output)  
            b[hidden_dim:] += alpha * delta_output  

            # Update weights and biases for hidden layer
            w += alpha * np.outer(data, delta_hidden) 
            b[:hidden_dim] += alpha * delta_hidden  

        mse /= len(train_data)  
        print(f'Epoch {epoch+1}, MSE: {mse:.4f}')
        epoch += 1

    return w, v, b, mse

def bp_test(w, v, b, test_data):
    z_in = np.dot(test_data, w) + b[:w.shape[1]]  
    z = sigmoid(z_in)                             
    y_in = np.dot(z, v) + b[w.shape[1]:]          
    y = sigmoid(y_in)                             
    return y

One Hot Encoder

In [140]:
def onehot_enc(lbl, min_val=0):
    mi = min(lbl)
    enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
    for i, x in enumerate(lbl):
        enc[i, x - mi] = 1
    return enc

def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]



Dataset Loading

In [141]:
from sklearn import datasets

# Load dataset and prepare data
iris = datasets.load_iris()
X = minmax_scale(iris.data)  # Normalize input features
Y = onehot_enc(iris.target)  # One-hot encode the target labels
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1, stratify=Y)

# Train the model
w, v, b, mse = bp_fit(X_train, y_train)

Epoch 1, MSE: 0.9729
Epoch 2, MSE: 0.6884
Epoch 3, MSE: 0.6779
Epoch 4, MSE: 0.6769
Epoch 5, MSE: 0.6761
Epoch 6, MSE: 0.6754
Epoch 7, MSE: 0.6747
Epoch 8, MSE: 0.6739
Epoch 9, MSE: 0.6732
Epoch 10, MSE: 0.6724
Epoch 11, MSE: 0.6716
Epoch 12, MSE: 0.6708
Epoch 13, MSE: 0.6699
Epoch 14, MSE: 0.6689
Epoch 15, MSE: 0.6679
Epoch 16, MSE: 0.6667
Epoch 17, MSE: 0.6653
Epoch 18, MSE: 0.6638
Epoch 19, MSE: 0.6620
Epoch 20, MSE: 0.6600
Epoch 21, MSE: 0.6576
Epoch 22, MSE: 0.6547
Epoch 23, MSE: 0.6514
Epoch 24, MSE: 0.6473
Epoch 25, MSE: 0.6425
Epoch 26, MSE: 0.6366
Epoch 27, MSE: 0.6297
Epoch 28, MSE: 0.6214
Epoch 29, MSE: 0.6118
Epoch 30, MSE: 0.6007
Epoch 31, MSE: 0.5883
Epoch 32, MSE: 0.5746
Epoch 33, MSE: 0.5601
Epoch 34, MSE: 0.5451
Epoch 35, MSE: 0.5299
Epoch 36, MSE: 0.5148
Epoch 37, MSE: 0.5002
Epoch 38, MSE: 0.4863


Prediction and Accuracy Report

In [142]:
predict = bp_test(w, v, b, X_test)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 66.67%
